### Advent of Code 2024: Day 22

In [1]:
import time
start_time = time.time()

In [2]:
from collections import defaultdict
from itertools import pairwise

def f(s):
    s ^= s << 6 & 0xFFFFFF
    s ^= s >> 5 & 0xFFFFFF
    return s ^ s << 11 & 0xFFFFFF

p1, p2 = 0, defaultdict(int)

for s in map(int, open("input.txt")):
    nums = [s] + [s := f(s) for _ in range(2000)]
    p1 += nums[-1]
    diffs = [b%10-a%10 for a,b in pairwise(nums)]
    seen = set()

    for i in range(len(nums)-4):
        pat = tuple(diffs[i:i+4])
        if pat not in seen:
            p2[pat] += nums[i+4] % 10
            seen.add(pat)

print(f"Part 1 solution: {p1}")
print(f"Part 2 solution: {max(p2.values())}")

Part 1 solution: 13004408787
Part 2 solution: 1455


In [3]:
end_time = time.time()
print(f"Execution time: {end_time - start_time:.3f} seconds")

Execution time: 4.404 seconds


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the data
df = pd.read_csv('nfl_play_by_play_data.csv')

# Data Preprocessing: Filter for end of game scores
end_game_scores = df[df['desc'] == "END GAME"]

# Extract relevant information
game_scores = end_game_scores[['game_id', 'total_home_score', 'total_away_score', 'home_team', 'away_team']]
game_scores['result'] = np.where(game_scores['total_home_score'] > game_scores['total_away_score'], 'home_win', 'away_win')

# Feature Engineering: Aggregate useful statistics for each game
# Example: Summarize stats such as total yards, turnovers, etc., per game
game_stats = df.groupby('game_id').agg({
    'yards_gained': 'sum',
    'pass_attempt': 'sum',
    'rush_attempt': 'sum',
    'turnover': 'sum',
    'score_differential': 'mean',
    # Add other relevant features as needed
}).reset_index()

# Merge the aggregated stats with the game results
game_data = game_scores.merge(game_stats, on='game_id')

# Prepare features and labels
X = game_data.drop(columns=['game_id', 'result', 'home_team', 'away_team'])
y = game_data['result']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Building: Use Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions and Evaluation
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Save the model (optional)
import joblib
joblib.dump(model, 'nfl_game_predictor.pkl')
